# Import Statements

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import OCR_pipeline as OCR
import TopicModeling as TM
import jsonpickle

# Cleaning And Saving Extracted Text

In [6]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="Axuan521",
  database="forestry"
)
mycursor = mydb.cursor()

def getExtractedTexts():
    mycursor.execute("SELECT filename, extractedText FROM PolicyPapers")
    myresult = mycursor.fetchall()
    return [x for x in myresult]

def getExtractedTextsFromCountries(countries):
    sqlStatement = "SELECT filename, extractedText FROM PolicyPapers WHERE"
    
    for i in range(len(countries)):
        whereClause = " country = '%s' " % countries[i]
        sqlStatement += whereClause
        if len(countries) != 1 and i < len(countries) - 1:
            sqlStatement += "or"
            
    mycursor.execute(sqlStatement)
    myresult = mycursor.fetchall()
    return [x for x in myresult]

In [7]:
#dictionary for texts in terms of sentences
import re
import gc
textDictionary = {}
countries = ['India']
policyPapers = getExtractedTextsFromCountries(countries)
for paper in policyPapers:
    cleanWhitespaces = re.sub(r'(.) ', r'\1', paper[1])
    cleanedText = TM.toSentences(cleanWhitespaces)
    textDictionary[paper[0]] = cleanedText
   # gc.collect()

# LDA

In [8]:
from pprint import pprint
lemmatized_sents = []

# Print the Keywords in the 5 topics
for key in textDictionary:
    cleanedText = textDictionary[key]
    lemmatized_sents.extend(cleanedText)
    
corpus, lda_model = TM.LDA(lemmatized_sents)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
#Next Step: Store LDA data in Policy Paper Object & other central file

[(3,
  '0.064*"indian" + 0.042*"table" + 0.032*"deposit" + 0.030*"security" + '
  '0.029*"bureau" + 0.027*"panchayat" + 0.026*"list" + 0.021*"document" + '
  '0.020*"gram" + 0.020*"mine"'),
 (13,
  '0.029*"shri" + 0.028*"probation" + 0.024*"sabha" + 0.022*"probationer" + '
  '0.018*"vacancy" + 0.017*"service" + 0.015*"manufacture" + 0.014*"procedure" '
  '+ 0.014*"represent" + 0.014*"vary"'),
 (19,
  '0.081*"disease" + 0.080*"estimate" + 0.036*"type" + 0.034*"lead" + '
  '0.030*"low" + 0.029*"opportunity" + 0.023*"small" + 0.021*"sl" + '
  '0.017*"sample" + 0.016*"spread"'),
 (17,
  '0.061*"monitoring" + 0.036*"index" + 0.035*"projection" + 0.034*"effort" + '
  '0.021*"current" + 0.020*"division" + 0.020*"especially" + 0.015*"regulate" '
  '+ 0.011*"march" + 0.011*"dept"'),
 (25,
  '0.061*"monsoon" + 0.038*"period" + 0.036*"range" + 0.034*"loss" + '
  '0.028*"likely" + 0.026*"damage" + 0.023*"cause" + 0.019*"fodder" + '
  '0.019*"requirement" + 0.019*"day"'),
 (9,
  '0.071*"animal" + 0

In [29]:
my = lda_model.print_topics()[0][1]

In [44]:
re.findall(r"[a-zA-Z]+",my)

['indian',
 'table',
 'deposit',
 'security',
 'bureau',
 'panchayat',
 'list',
 'document',
 'gram',
 'mine']

## Extract sentences by Keywords

### TODO: Have a feature on the web app so that the user can extract text using keywords

In [ ]:
#Try with the first document
policy_0 = policyPapers[0][1]
sentences = TM.ReturnSentence(policy_0)

In [73]:
#This can be used as a preview feature
for s in sentences:
    if 'land' in s:
        print(s)

 (2) Notwithstanding anything contained in sub-rule (1) a scheme of prospecting, shall include: (a) baseline information of prevailing environmental conditions before the beginning of prospecting operations; (b) [steps proposed to be taken for protection of environment which will include] prevention and control of air and water pollution, progressive reclamation and rehabilitation of the land disturbed by the prospecting operations, a scheme for the plantation of trees, and such other measures, as may be directed from time to time by the Controller General or the authorised officer for minimising the adverse effect of prospecting operations on the environment
 Types of plans and sections: (1) The owner, agent, mining engineer or manager of every mine shall keep the following plans and sections(a) a surface plan showing every surface feature within the mining lease boundaries, such as building, telephone, telegraph or power transmission line, watermain, tramline, railway, road, river, w

## Use Key Words to classify documents

In [48]:
key_words = ['afforestation','agriculture','animal welfare','artificial regeneration','biodiversity',
             'biological resources','biome','board','clean','coconut','conservation','control','database',
             'enforcement','environment','farm','financing','forest','forest protection','funding','fundraising',
             'land ','land use','landholder','law','measuring','mobilization','monitor','natural resources',
             'oversight','plant breeders','plants','pollution','preservation','produce','protection','qualification',
             'quality','registry','regulation','reporting','reserve','resource','restriction','results-driven',
             'rural','safeguard','species' ,'support','sustainable','technical submission','threatened species',
             'variety','verification','wastewater','water','watershed','wild life']

In [58]:
'artificial regeneration' in cleanWhitespaces

False

In [68]:
# Create a list with each entry being whether or not containing the keyword
if_key_word = [int(word in cleanWhitespaces) for i, word in enumerate(key_words)]

In [69]:
if_key_word

[0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0]

# Visualizations

### pyLDAvis

`pip install pyldavis`

Code source: https://markroxor.github.io/gensim/static/notebooks/gensim_news_classification.html
<br>
The area of the circles represent the prevalance of the topic. The length of the bars on the right represent the membership of a term in a particular topic. 

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)

### WordCloud

`pip install wordcloud`

Code source: https://medium.com/@rudyb2001/making-sense-of-the-news-using-machine-learning-e13ed5f96cf0
<br>
WordCloud of Top 10 words in each topic with the sizes of the words proportional to the weights.

In [ ]:
len(lda_model.show_topics(formatted=False))

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from wordcloud import WordCloud, STOPWORDS
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]

cloud = WordCloud(background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 5, figsize=(15,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

# Storage